In [3]:
import torch
from torch import nn
from torch.utils.data import DataLoader # dataset을 순회가능한 객체(iterable)
from torchvision import datasets # 샘플(data) and labels
from torchvision.transforms import ToTensor

In [4]:
# 공개 데이터 셋에서 학습 데이터 내려받기
training_data = datasets.FashionMNIST(
    root ='data',
    train = True,
    download = True,
    transform = ToTensor(),    
)

# 공개 데이터 셋에서 테스트 데이터 내려받기
test_data = datasets.FashionMNIST(
    root ='data',
    train = False,
    download = True,
    transform = ToTensor(),    
)

Dataset을 DataLoader의 인자로 전달 <br>
이는 데이터셋을 순회 가능한 객체(iterable)로 감싸고, 자동화된 배치(batch), 샘플링(sampling), 섞기(shuffle) 및 다중 프로세스로 데이터 불러오기(multiprocess data loading)를 지원  <br>
즉, 데이터로더 객체의 각 요소는 64개의 feature와 label을 묶음(batch)으로 반환

In [5]:
batch_size = 64

# 데이터로더 생성
train_dataloader = DataLoader(training_data, batch_size = batch_size)
test_dataloader = DataLoader(test_data, batch_size = batch_size)

for X, y in test_dataloader :
    print("Shape of X [N, C, H, W] : {}".format(X.shape))
    print("Shape of y : {}".format(y.shape))
    break

Shape of X [N, C, H, W] : torch.Size([64, 1, 28, 28])
Shape of y : torch.Size([64])


모델 만들기! <BR>
파이토치에서 신경망 모델은 nn.Module을 상속받는 클래스를 생성하여 정의 <br>
__init__ 함수에서 신경망의 계층(layer)들을 정의하고 forward 함수에서 신경망에 데이터를 어떻게 전달할지 지정

nn.Module을 상속한 subclass가 신경망 모델로 사용되기 위해선 앞서 소개한 두 메소드를 override 해야한다. <br>

__init__(self): initialize; 내가 사용하고 싶은, 내 신경망 모델에 사용될 구성품들을 정의 및 초기화 하는 메소드이다. <br>
forward(self, x): specify the connections;  이닛에서 정의된 구성품들을 연결하는 메소드이다. 

<flatten 예시> <br>
input_image = torch.rand(64, 28, 28)   # torch.Size([64,28,28])<br>
flatten = nn.Flatten()<br>
flat_image = flatten(input_image)  # torch.Size([64, 784])

In [6]:
# 학습에 사용할 CPU나 GPU 장치 얻기
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

print("Using {} device".format(device))

# 모델 저의하기
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__() # 부모 클래스인 nn.Module을 super()을 사용해서 초기화
        self.flatten = nn.Flatten() # 연속된 범위를 가진 차원을 하나의 텐서로 Flatten해준다.
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
        
    def forward(self, x):
        x = self.flatten(x)
        logists = self.linear_relu_stack(x)
        return logists

model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


모델 매개변수 최적화하기 <br>
모델을 학습하려면 loss function과 옵티마이저가 필요

In [7]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 1e-3)

각 학습 단계(training loop)에서 모델은 (배치로 제공되는) 학습 데이터셋에 대한 예측을 수행하고, 예측 오류를 역전파하여 모델의 매개 변수를 조정

pytorch에서는 왜 항상 optimizer.zero_grad()를 해줄까? <br>

Pytorch에서는 gradients값들을 추후에 backward를 해줄때 계속 더해주기 때문"에 우리는 항상 backpropagation을 하기전에 gradients를 zero로 만들어주고 시작을 해야합니다.

In [8]:
# train 함수는 nn.Module 바깥에 지정해주는구나
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # 예측 오류 계산
        pred = model(X)
        loss = loss_fn(pred, y)

        # 역전파
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
        

테스트

In [9]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval() # 모델을 평가 모드로 설정
    test_loss, correct = 0, 0
    with torch.no_grad(): #  안에서 계산된 그래디언트가 모델 가중치에 업데이트되지 않도록 합니다. 이것은 테스트 중에는 그래디언트가 필요하지 않기 때문
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()                            # item() 함수는 PyTorch 텐서(tensor)에서 스칼라 값을 추출하는 데 사용되는 메서드
            correct += (pred.argmax(1) == y).type(torch.float).sum().item() # pred.argmax(1) 가장 큰 값을 반환 
    test_loss /= num_batches
    correct /= size
    print(size)
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [10]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------


loss: 2.298333  [   64/60000]
loss: 2.285150  [ 6464/60000]
loss: 2.267569  [12864/60000]
loss: 2.262990  [19264/60000]
loss: 2.247089  [25664/60000]
loss: 2.220218  [32064/60000]
loss: 2.233292  [38464/60000]
loss: 2.201061  [44864/60000]
loss: 2.195114  [51264/60000]
loss: 2.163738  [57664/60000]
10000
Test Error: 
 Accuracy: 45.4%, Avg loss: 2.150543 

Epoch 2
-------------------------------
loss: 2.161657  [   64/60000]
loss: 2.148637  [ 6464/60000]
loss: 2.090363  [12864/60000]
loss: 2.105748  [19264/60000]
loss: 2.050162  [25664/60000]
loss: 1.998369  [32064/60000]
loss: 2.029078  [38464/60000]
loss: 1.953416  [44864/60000]
loss: 1.956926  [51264/60000]
loss: 1.883521  [57664/60000]
10000
Test Error: 
 Accuracy: 55.9%, Avg loss: 1.874129 

Epoch 3
-------------------------------
loss: 1.908821  [   64/60000]
loss: 1.876827  [ 6464/60000]
loss: 1.753479  [12864/60000]
loss: 1.795454  [19264/60000]
loss: 1.680299  [25664/60000]
loss: 1.641784  [32064/60000]
loss: 1.666304  [38464/6

str